# https://github.com/kernoelpanic/sccc

# Smart Contract Crash Course

This is an introduction to Smart Contracts based on Ethereum, Ganache and Web3py.
This tutorial is is part of the [Cryptocurrencies lecture](https://tiss.tuwien.ac.at/course/courseDetails.xhtml?dswid=1923&dsrid=980&courseNr=192065&semester=2018W) at TU Wien.

## Informal introduction to Smart Contracts

* Concept/idea of smart contracts dates back to the 90s 
* Original definition is attributed to [Nick Szabo 1994](https://web.archive.org/web/20160323035617/http://szabo.best.vwh.net/smart.contracts.html):
> “A smart contract is a **computerized transaction
protocol that executes the terms of a contract**. The
general objectives are to satisfy common contractual
conditions (such as payment terms, liens, confidentiality,
and even enforcement), **minimize exceptions both
malicious and accidental**, and **minimize the need for
trusted intermediaries**. Related economic goals include
lowering fraud loss, arbitrations and enforcement costs,
and other transaction costs.”

### Let's envision such a platform, what are the challenges?

* **Goals:**
    + Decentralized smart contract platform
    + Verify the correct execution of each others code
        - same code was executes
        - the result/output is correct

* **Requirements:**
    + Set of nodes/participants
    + Network connection (path to every node)
    + Code (boradcast and store on every node)
    + Input data (send to every node)

* **What can possibly go wrong?:**
    + Remote code execution vulnerability per design
        - Software security aspects
        - Sandbox/VM required
        - In Ethereuem: 
            - EVM (Ethereuem Virtual Machine)
            - no disk I/O
            - no network connection
            - only update to state variables
    + Halting problem
        - Need to limit the number of execution steps
        - In Ethereuem:
            - Pay *Gas* for every execution step
    + Correctness of the deployed code 
        - Software security aspects ...
        - In Etheruem:
            - Fail => potentialy lose funds 
    + Network attacks (MitM, eclipse attacks, DDoS, BGP attacks, ...)
        - Network security aspects, cryptography aspects, ...
        - Ensure integrity and authenticity of sent transactions
        - In Etheruem:
            - Transactions contain cryptographic signature of sender
    + Need to agree on total order of executions => consensus required
        - Distributed systems aspects, cryptography aspects, economic aspects, ...
        - In Etheruem:
            - Currently Nakamoto Consensus 

## Ethereum

Ethereum is a smart contract platform, there are also others (e.g., [Cardano](https://www.cardano.org/en/home/),[hyperleder](https://www.hyperledger.org/),[rootstock](https://www.rootstock.com/),[quorum](https://github.com/jpmorganchase/quorum), ...) but Ethereum is the most common (and I dont know as much about the others yet :) Therefore the tutorial as well as the project/exercise will focus on Ethereum style smart contracts. 

**Note:** A lot will (or is supposed to) change in future ethereum versions i.e., [ETH2.0](https://docs.ethhub.io/ethereum-roadmap/ethereum-2.0/eth-2.0-phases/). We focus currently on what is running at the moment i.e., ETH1.x



### Hard Facts

* Ethereum is a Cryptocurrency and smart contract platform proposed 2013 by Vitalik Buterin and Gavin Wood
* Currency symbol: ETH
* Currency name: **ether**
    + smallest unit: **wei**
    + 1 ether = 10^18 wei
    + Beware: All currency units are per default in wei!
* Technically Ethereum EVM is not Turing Complete because of Gas limits *but it doesn’t matter anyway* [1](https://media.consensys.net/ethereum-isnt-turing-complete-and-it-doesn-t-matter-anyway-625061294d3c) [2](https://www.youtube.com/watch?v=cGFOKTm_8zk)
* Solidity high-level language, syntax comparable to JavaScript
    + compiles to EVM byte code 
    + executed in EVM (Ethereum Virtual Machine)

### Overview

Etheruem in [one picture](https://github.com/kernoelpanic/BlockchainIllustrations/tree/master/Ethereum) (slightly outdated but still quite close):

<img src="https://raw.githubusercontent.com/kernoelpanic/BlockchainIllustrations/master/Ethereum/EthBlockchain5_bg.png" width=1024 height=768>

### Account model
In contrast to other Cryptocurrencies like Bitcoin (UTXO), Ethereum has the concept of **accounts** i.e., state which holds the current balance of all addresses. 

There are two types of accounts:
* **Externally controlled** accounts (aka. normal accounts)
    + Has an associated private/public key pair, like in Bitcoin
        - Address consists of last 20 bytes of public key
        - `${Keccak­256( pk ) ):96:255}`
    + Controlled by their private keys with the respective wallet/client 
* **Contract accounts**
    + Has associated code, which is immutable after initialization (`codeHash`)
    + Has associated state (`storageRoot`)
    + Has **no** associated private/public key pair
    + Entirely controlled by its code 
    + Address is deterministically generated based on
        - address of sender and how many transactions/contract creations made so far i.e., nonce
        - or code of contract (create2)
    
Moreover, each account (external and contract) has:
* `address`: 20 byte hex string, also immutable
* `balance`: Scalar value equal to the number of Wei owned by this account
* `nonce`: Scalar value equal to the number of transactions sent from this external account or, number of contract-creations made by this contract. This value is for replay protection.

### Terminology and important references 

#### Ethereum(s)
There are several Ethereum(s) (due to community disagreement about the DAO Hack), but those are (still) technically more or less the same, dispite their blockchains differ since the DAO Hack : 
* [Ethereum](https://www.ethereum.org/)
    - we will refer to this Ethereum for the rest of the lecture
    - [wiki](https://eth.wiki/en/home)
    - [ethhub](https://docs.ethhub.io/)
    - [yellow paper](https://ethereum.github.io/yellowpaper/paper.pdf)
    - [yellow paper github](https://github.com/ethereum/yellowpaper) (really the current version)
    - [white paper](https://eth.wiki/en/white-Paper) (more decent introduction)
* [Ethereum Classic](https://ethereumclassic.github.io/)

#### Versions / Roadmap
The major versions of the Ethereum specification have dedicated names like: *Frontier*, *Homestead*, *Metropolis*, *Byzantium*, *Constantinople*, *Istanbul*,  ... 
* [Roadmap](https://eth.wiki/en/roadmap)

#### Clients
There are also several Ethereum clients e.g.,:
* [geth](https://github.com/ethereum/go-ethereum/wiki/geth)
    - written in go
    - we will use this client in the project/exercise
* [parity](https://github.com/paritytech/parity-ethereum)
    - written in rust
* [cpp-ethereum](https://github.com/ethereum/aleth/wiki)
    - written in cpp
* [pyethapp](https://github.com/ethereum/pyethapp)
    - written in python
    - mostly for dev. purposes

#### Testnets
There are several Ethereum networks / blockchains.

* Mainnet
    - web blockchain explorers:
        - [Etherscan](https://etherscan.io/)
        - [Etherchain](https://www.etherchain.org/)
* Public Testnets e.g.,:
    - *Ropsten* 
        - PoW, should be compatiple with all clients that also work on main chain
        - [Etherscan Ropsten explorer](https://ropsten.etherscan.io/)
    - *Morden*
        - Old public PoW testnet, now Ethereum classic testnet
    - *Rinkeby*
        - Public PoA geth compatible testnet
    - *Kovan*
        - Public PoA parity compatible testnet 
    
There are also private testnets. For the project/exercise we will use a
private geth PoA network to which you will connect your nodes. 

For the tutorial we will use a small development testnet provided by `ganache`

## Tutorial on Smart Contracts in Ethereuem 

### Basic interaction with `geth`,`parity` or `ganache` using web3py

The most common client lib to interface with ethereum clients (`geth`,`ganache`,...) is **web3**.
* [web3.js](https://github.com/ethereum/web3.js/)
    - node version
* [web3py](https://github.com/ethereum/web3.py)
    - python version
    - as I prefer python, we will use this one ;)
    - https://web3py.readthedocs.io/en/stable/

For the tutorial we will only interact with `ganache` which uses his own local development testnet blockchain. But all commands issues via `web3py` are the same when connected to a "real" `geth` or `parity` node.
* [ganache-cli](https://github.com/trufflesuite/ganache-cli/tree/master)

When running `geth` yout can also attach to it and use the JavaScript console (Read, Evaluate & Print Loop = REPL). 
Some stuff works in the geth console that does not work in `web3*`.
Once `geth` is running on localhost start the binary again to attach to the running instance:
```
geth attach "http://localhost:8545"
```

### Connect
Import the web3 libarary and connect to a running `geth`/`parity` or `ganache` node via `HTTPProvider`

For the tutorial configuration the docker container of ganache is located at `172.18.0.2`

In [31]:
import web3

In [32]:
w3 = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.2:8545"))
# check if connection was successful
assert w3.isConnected()

If connected to `geth`,`parity` or as in our case to `ganache` you can execute serveral commands with the client to interact with the blockchain the client is sychronizing to. 



First it is always good the check to which blockchain the client is connected. In our case `ganache` starts a development chain with a random ID. The network ID provides information on which network the connected node resides.
The Ethereuem main network has id `1`.
Public test networks or private custom test networks should have different network IDs.
For a list of network IDs see this list:
* https://ethereum.stackexchange.com/questions/17051/how-to-select-a-network-id-or-is-there-a-list-of-network-ids

**Note:** Ethereuem and Ethereuem Classice have the same network ID! To avoid replay attacks of transactions the `CHAIN_ID` was introduced and is used in transactions (encoded into the signature creation, see below). 
* https://github.com/ethereum/EIPs/blob/master/EIPS/eip-155.md

In [33]:
# display network ID of client you are connected to
w3.net.version

'1575555942207'

Next it is interesting if the node is connected to other peers e.g., other `geth` clients.

For our tutorial `ganache` is not connected to any peer since we have our own local blockchain. Our w3 connection is not a peer! The w3 connection is just our interface to our client, therefore only local.

For the exercise your `geth`/`parity` node will connect to our server in the private PoA testnet. Then you should see at least one peer i.e., our server. For a connection to be successful there are two requirements:
* Both peers need to have the same **network ID** 
* Both peers need to have the same **genesis.json** / genesis block


In [34]:
# number of connected peers to our peer
w3.net.peerCount

0

For the exercise it will also be important to chech if your node is synchronized with the head of the blockchain i.e., our PoA node. 
If the node is started for the first time he will after some time start syncing. 
If syncing is *False* then you should see a `blockNumber` > 0

In [35]:
# check if node is "up-to-date" with blockchain, 
# i.e., not syncing anymore
w3.eth.syncing

False

In [36]:
# return current blockchain head of node
# Since ganache "simulates" blokchain,
# blocks are mined on demand.
# Therefore, no block has been mined yet
w3.eth.blockNumber

0

In [42]:
# block 0 ist the genesis block
genesis_block = w3.eth.getBlock(0)
genesis_block

AttributeDict({'number': 0,
 'hash': HexBytes('0xe203965c9eace3138398f707ea080ddb2dc636e5cda151e8a5b1f2d33370e96b'),
 'parentHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

As you see, the genesis block is special as it is the only block with no parent hash:

In [44]:
genesis_block['number']

0

In [43]:
genesis_block['parentHash']

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')

In [38]:
# instruct our test blockchain to mine a block
w3.provider.make_request('evm_mine',params='')

{'id': 6, 'jsonrpc': '2.0', 'result': '0x0'}

In [39]:
# for older web3py versions the command would be:
# w3.providers[0].make_request('evm_mine',params='')

In [40]:
w3.eth.blockNumber

1

In [45]:
first_block = w3.eth.getBlock(1)
first_block

AttributeDict({'number': 1,
 'hash': HexBytes('0xa9e4072d4b8bb2905fa91128596616d93bbbeba19972e4228cbb6a27032c7a94'),
 'parentHash': HexBytes('0xe203965c9eace3138398f707ea080ddb2dc636e5cda151e8a5b1f2d33370e96b'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

In [46]:
first_block['number']

1

In [47]:
first_block['parentHash']

HexBytes('0xe203965c9eace3138398f707ea080ddb2dc636e5cda151e8a5b1f2d33370e96b')

In [48]:
# our node is configured to mine blocks 
w3.eth.mining

True

In [49]:
# but since it is a fake chain we have no hashrate
w3.eth.hashrate

0

### Accounts and balances

Accounts are your **external accounts** consisting of public and private keys. 
The ethereum address are the last 40 hex characters (20 bytes) of the hash (Keccak-256) of the public key prefixed with `0x`

`ganache` generates a list of accounts on startup. 
For the exercise you will recieve your personal account keypair which you have to copy in your `geth`/`parity` folder to import it. 

In [50]:
w3.eth.accounts

['0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 '0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0',
 '0x22d491Bde2303f2f43325b2108D26f1eAbA1e32b',
 '0xE11BA2b4D45Eaed5996Cd0823791E0C93114882d',
 '0xd03ea8624C8C5987235048901fB614fDcA89b117',
 '0x95cED938F7991cd0dFcb48F0a06a40FA1aF46EBC',
 '0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9',
 '0x28a8746e75304c0780E011BEd21C72cD78cd535E',
 '0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E',
 '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e']

In [65]:
print("address length with 0x at the beginning = ",len(w3.eth.accounts[0]))
w3.eth.accounts[0]

address length with 0x at the beginning =  42


'0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'

**Note:** Ethereum uses Keccak-256 but often refers to it as SHA3, but the output of SHA3 as specified by NIST is different!

In [51]:
from sha3 import sha3_256
sha3_256(b'').hexdigest()

'a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a'

In [52]:
from sha3 import keccak_256
keccak_256(b'').hexdigest()

'c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'

In [59]:
w3.sha3(text="").hex()[2:]

'c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'

In [60]:
assert keccak_256(b'').hexdigest() == w3.sha3(text="").hex()[2:]

The default address (also used to receive rewards from mining blocks) is usually the first in the list of accounts and usually also accessible via `w3.eth.coinbase`

In [66]:
assert w3.eth.accounts[0] == w3.eth.coinbase
w3.eth.coinbase

'0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'

There is also a helper method to check if a string is an ethereum address.

In [30]:
w3.isAddress(w3.eth.accounts[0])

True

In [67]:
test_addr = '0xd3cda913deb6f67967b99d67acdfa1712c293601'
print(test_addr,"\n",len(test_addr))
print(test_addr[:-1],"\n",len(test_addr[:-1]))
w3.isAddress(test_addr[:-1])

0xd3cda913deb6f67967b99d67acdfa1712c293601 
 42
0xd3cda913deb6f67967b99d67acdfa1712c29360 
 41


False

In [68]:
w3.isAddress(test_addr)

True

**Note:** The capital letters of an address encode an optional checksum
https://github.com/ethereum/EIPs/blob/master/EIPS/eip-55.md

In [69]:
w3.isChecksumAddress(test_addr)

False

In [71]:
test_addr = w3.toChecksumAddress(test_addr)
print(test_addr)
w3.isChecksumAddress(test_addr)

0xd3CdA913deB6f67967B99D67aCDFa1712C293601


True

### Example: Create externally owned account 

To see what an externally controlled account actually is, lets create one. 
First useing `geth` then with `web3.py`.

In [124]:
!pwd 

/smartcode


In [126]:
!ls -lad examples

drwxr-xr-x 2 smartcode smartcode 4096 Dec  4 05:04 examples


In [129]:
!mkdir ./examples/geth/

In [130]:
!echo "password" > ./examples/pwd.txt

In [131]:
!geth --maxpeers 0 --datadir=./examples/geth --password=./examples/pwd.txt account new

INFO [12-09|13:21:47.449] Maximum peer count                       ETH=0 LES=0 total=0
INFO [12-09|13:21:47.449] Smartcard socket not found, disabling    err="stat /run/pcscd/pcscd.comm: no such file or directory"

Your new key was generated

Public address of the key:   0x202d3601c9675b0E5df92fa748b0BFE715Fb94f3
Path of the secret key file: examples/geth/keystore/UTC--2019-12-09T13-21-47.449873923Z--202d3601c9675b0e5df92fa748b0bfe715fb94f3

- You can share your public address with anyone. Others need it to interact with you.
- You must NEVER share the secret key with anyone! The key controls access to your funds!
- You must BACKUP your key file! Without the key, it's impossible to access account funds!
- You must REMEMBER your password! Without the password, it's impossible to decrypt the key!



In [154]:
keystore_file = !ls ./examples/geth/keystore
keystore_file[0]

'UTC--2019-12-09T13-21-47.449873923Z--202d3601c9675b0e5df92fa748b0bfe715fb94f3'

In [156]:
with open('./examples/geth/keystore/' + keystore_file[0]) as keyfile:
    keyfile_json = keyfile.read()

keyfile_json

'{"address":"202d3601c9675b0e5df92fa748b0bfe715fb94f3","crypto":{"cipher":"aes-128-ctr","ciphertext":"b7f696b18d78103c28a02d40b0bd4299df61e3299620a279f7a26c76140225a5","cipherparams":{"iv":"2c1d8d928b987ace07d05f2dc9cd9122"},"kdf":"scrypt","kdfparams":{"dklen":32,"n":262144,"p":1,"r":8,"salt":"7491eb49a0b73026e63d8d05a7e93b48bf3119d8c9217d0988ff017d8639c385"},"mac":"87ff1b8430b2cb3b3c36df072dc812eba6d2a1148ccd05a019ffcc43ad20be20"},"id":"f8b295c1-41c4-4959-8dc6-62948dfb6289","version":3}'

In [157]:
private_key = w3.eth.account.decrypt(keyfile_json,"password")

In [158]:
private_key.hex()

'0xf1fce73f5ef3ea33825e601a46d1d1fd1a747fb149d6e939db8202b828f176fa'

In [161]:
account = w3.eth.account.privateKeyToAccount(private_key)

In [162]:
account.address

'0x202d3601c9675b0E5df92fa748b0BFE715Fb94f3'

In [164]:
account.privateKey.hex()

'0xf1fce73f5ef3ea33825e601a46d1d1fd1a747fb149d6e939db8202b828f176fa'

Lets create an account directly with web3.py:

In [148]:
account = w3.eth.account.create("random string to increase entropy")

In [149]:
account.address

'0x503A87e5f87141F6c677eBf5A9dc6d6a0ec813CA'

In [150]:
account.privateKey

HexBytes('0xb11f2756db6b9c55113ef141735ff628d723beaab6b1b937bb1e7fe60f4c03d8')

### Balance and gas
To query the balance of an account or contract the function `eth.getBalance()` can be used with the respective address.

**Note:** Per default all balances are in *Wei* i.e., the smallest unit of value in Ethereum. 
$$
    1 \textrm{ ether} = 10^{18} \textrm{ wei} 
$$

In [72]:
balance = w3.eth.getBalance(w3.eth.accounts[0])
balance

100000000000000000000

In [73]:
balance * 10**-18

100.00000000000001

In [74]:
# Directly output the balance in ether
w3.fromWei(balance,'ether')

Decimal('100')

In [75]:
def getBalance(address):
    return w3.fromWei(w3.eth.getBalance(address),'ether')

In [76]:
getBalance(w3.eth.accounts[0])

Decimal('100')

In [77]:
getBalance(w3.eth.accounts[1])

Decimal('100')

We now send a transaction that just transfers value i.e., ether from one account to another. Thereby, `ganache` automatically mines a block once he sees that a transactions requires confirmation. 

**Note:** When connected to `geth` or `parity` you have to unlock an account first to send a transaction since the private keys are stored encrypted per default and protected with a password. In `ganache` this is not necessary since it is a development environment.

In [80]:
# required in geth but not in ganache
# Parameters are:
# * account
# * password
# * duration of unlock (if 0 then forever)
w3.geth.personal.unlockAccount(w3.eth.accounts[0],"",0)

True

In [81]:
# old general way now client may be different.
#w3.personal.unlockAccount(w3.eth.accounts[0],"",0)

In [82]:
w3.eth.blockNumber

1

In [83]:
w3.eth.sendTransaction({'from':w3.eth.accounts[0], 
                        'to':w3.eth.accounts[1], 
                        'value':10**18})

HexBytes('0xbfda5a53139f0066893d3d3947df833807b90b0ab567494d1cc21b0c42b7b04c')

In [84]:
blkNumber = w3.eth.blockNumber
blkNumber

2

In [90]:
# the balance is reduced by more than 1 ether
# Why do you think this is the case?
getBalance(w3.eth.accounts[0])

Decimal('98.99958')

In [167]:
getBalance(w3.eth.accounts[1])

Decimal('101')

In [172]:
# get first (and only) transaction in the previously mined block
tx = w3.eth.getTransactionByBlock(blkNumber,0)
tx 

AttributeDict({'hash': HexBytes('0xbfda5a53139f0066893d3d3947df833807b90b0ab567494d1cc21b0c42b7b04c'),
 'nonce': 0,
 'blockHash': HexBytes('0x28d5623c6cf6a3fdeb1afdc09f57dbc57a6aaa929e835e8a8c2f56652f6d05ee'),
 'blockNumber': 2,
 'transactionIndex': 0,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0',
 'value': 1000000000000000000,
 'gas': 121000,
 'gasPrice': 20000000000,
 'input': '0x',
 'v': 28,
 'r': HexBytes('0x7c95c2cac783bb99be1314d3f3fbf41d1b3befb550d9c469e2378ce9a537d942'),
 's': HexBytes('0x6cdc83acacfd8d71721df2e24907db8bb04fc81306cc4922b818b294e8732eaf')})

In [169]:
# get block and see the transaction id and gas used below
blk = w3.eth.getBlock(blkNumber)
blk

AttributeDict({'number': 2,
 'hash': HexBytes('0x28d5623c6cf6a3fdeb1afdc09f57dbc57a6aaa929e835e8a8c2f56652f6d05ee'),
 'parentHash': HexBytes('0xa9e4072d4b8bb2905fa91128596616d93bbbeba19972e4228cbb6a27032c7a94'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

In [182]:
# get the transaction ID of the first (and only) transaction in the block
blk["transactions"][0].hex()

'0xbfda5a53139f0066893d3d3947df833807b90b0ab567494d1cc21b0c42b7b04c'

In [178]:
tx["hash"].hex() # hash of the transaction

'0xbfda5a53139f0066893d3d3947df833807b90b0ab567494d1cc21b0c42b7b04c'

In [183]:
# Query the transaction by ID, also possible
w3.eth.getTransaction(blk["transactions"][0].hex())

AttributeDict({'hash': HexBytes('0xbfda5a53139f0066893d3d3947df833807b90b0ab567494d1cc21b0c42b7b04c'),
 'nonce': 0,
 'blockHash': HexBytes('0x28d5623c6cf6a3fdeb1afdc09f57dbc57a6aaa929e835e8a8c2f56652f6d05ee'),
 'blockNumber': 2,
 'transactionIndex': 0,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0',
 'value': 1000000000000000000,
 'gas': 121000,
 'gasPrice': 20000000000,
 'input': '0x',
 'v': 28,
 'r': HexBytes('0x7c95c2cac783bb99be1314d3f3fbf41d1b3befb550d9c469e2378ce9a537d942'),
 's': HexBytes('0x6cdc83acacfd8d71721df2e24907db8bb04fc81306cc4922b818b294e8732eaf')})

In [170]:
blk["gasUsed"] # gas used within block

21000

In [49]:
# estimate the gas price of an transaction without executing it
w3.eth.estimateGas({'from':w3.eth.accounts[0], 
                    'to':w3.eth.accounts[1], 
                    'value':10**18})

21000

The estimate yields the same result as the actual gas cost. 
Can this be different?

In [180]:
tx["gasPrice"] # The actual gas price attached to the tx

20000000000

In [181]:
# conversion between gas and ether specified per transaction
# i.e., how much wei a transaction is willing to pay for 
w3.eth.gasPrice # default value

20000000000

In [51]:
# gasUsed(by the transaction) * gasPrice(specified in tx)
# gives the wei it costs to send it 
# times 10^18 gives the ether
(21000 * 20000000000) * 10**-18

0.00042

This is exaclty the missing cost on the sender side: 

In [189]:
float(getBalance(w3.eth.accounts[0]))

98.99958

In [187]:
float(getBalance(w3.eth.accounts[0])) + 0.00042

99.0

## Contract deployment and interaction (the hard way)

Now its time to deploy our first contract.
For the first time we do it the hard way.

In [193]:
# execute console command and print Greeter.sol
!cat ./examples/Greeter/Greeter.sol

pragma solidity ^0.5.12;

contract Greeter {
    string public greeting;
    uint256 public greetbit;

    //Old constructor syntax, function name same as contract name:
    //function Greeter() public {
    //new constructor syntax:
    constructor() public {
        greeting = 'Hello';
    }

    function setGreeting(string _greeting) public {
        greeting = _greeting;
    }

    function greet() view public returns (string) {
        return greeting;
    }

    function setGreetbit(uint256 bit) public {
        greetbit = bit;
    }

    function greetbit() view public returns (uint256) {
        return greetbit; 
    }

    function() public payable{
        greetbit = greetbit ^ 1;
    }

}


In [195]:
cd ./examples/Greeter 

/smartcode/examples/Greeter


In [196]:
!pwd

/smartcode/examples/Greeter


In [198]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.5.12+commit.7709ece9.Linux.g++


In [197]:
!solc --help

solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if none given or
"-" is used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also possible to
remap paths using the context:prefix=path syntax.
Example:
solc --bin -o /tmp/solcoutput dapp-bin=/usr/local/lib/dapp-bin contract.sol

Allowed options:
  --help               Show help message and exit.
  --version            Show version and exit.
  --license            Show licensing information and exit.
  --evm-version version
                       Select desired EVM version. Either homestead, 
                       tangerineWhis

In [203]:
!solc --bin --overwrite -o ./ Greeter.sol

Compiler run successful. Artifact(s) can be found in directory ./.


In [204]:
!cat Greeter.bin

608060405234801561001057600080fd5b506040518060400160405280600581526020017f48656c6c6f0000000000000000000000000000000000000000000000000000008152506000908051906020019061005c929190610062565b50610107565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106100a357805160ff19168380011785556100d1565b828001600101855582156100d1579182015b828111156100d05782518255916020019190600101906100b5565b5b5090506100de91906100e2565b5090565b61010491905b808211156101005760008160009055506001016100e8565b5090565b90565b610529806101166000396000f3fe6080604052600436106100555760003560e01c80631c55ad30146100625780634f11dbea1461008d578063a4136862146100c8578063a42af43f14610190578063cfae3217146101bb578063ef690cc01461024b575b6001805418600181905550005b34801561006e57600080fd5b506100776102db565b6040518082815260200191505060405180910390f35b34801561009957600080fd5b506100c6600480360360208110156100b057600080fd5b81019080803590602001909291905050506102e1565b005b3480156100d457600080fd5b5061018e6004803603

In [205]:
# assign cat output to python variable
out = !cat Greeter.bin
cbin = out[0]

In [206]:
!solc --abi --overwrite -o ./ Greeter.sol

Compiler run successful. Artifact(s) can be found in directory ./.


In [207]:
!cat Greeter.abi

[{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":true,"stateMutability":"payable","type":"fallback"},{"constant":true,"inputs":[],"name":"getGreetbit","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"greetbit","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"greeting","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"uint256","name":"_bit","type":"uint256"}],"name":"setGreetbit","outputs":[],"payable":false,"stateMutability":"nonpayable

In [208]:
out = !cat Greeter.abi
cabi = out[0]

One way to send a deploy transaction is using the w3py libary helper function cunstructor.

In [209]:
# send transaction to deploy contract with helper function
contract=w3.eth.contract(abi=cabi,
                         bytecode=cbin)
tx_hash = contract.constructor().transact({"from":w3.eth.accounts[0],})
tx_hash

HexBytes('0xe97f2dcc454c136a3f245475e61662a8a4c512f718bcbc702b33cafebc9714e4')

Another way which can also estimate the gas costs of the deploy transaction is sending is 
directly via `sendTransaction` and add the previously compiled contract as `data`

In [210]:
# estimate gas of deployment transaction
w3.eth.estimateGas({"from":w3.eth.accounts[0],"data":cbin,"value":0})

439932

In [211]:
# send transaction and get transaction hash
tx_hash = w3.eth.sendTransaction({"from":w3.eth.accounts[0],"data":cbin,"value":0})
tx_hash

HexBytes('0x10ea154a3dc212e1dbc57c63cc23a5758258ff0c17f8af4db802b14995e0dc31')

In [212]:
tx_info = w3.eth.getTransaction(tx_hash)
tx_info

AttributeDict({'hash': HexBytes('0x10ea154a3dc212e1dbc57c63cc23a5758258ff0c17f8af4db802b14995e0dc31'),
 'nonce': 2,
 'blockHash': HexBytes('0x2c0e4802f531edef175a38bf0283b778b9f5725972ae9fdd9ed5844aa494e880'),
 'blockNumber': 4,
 'transactionIndex': 0,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': None,
 'value': 0,
 'gas': 539932,
 'gasPrice': 20000000000,
 'input': '0x608060405234801561001057600080fd5b506040518060400160405280600581526020017f48656c6c6f0000000000000000000000000000000000000000000000000000008152506000908051906020019061005c929190610062565b50610107565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106100a357805160ff19168380011785556100d1565b828001600101855582156100d1579182015b828111156100d05782518255916020019190600101906100b5565b5b5090506100de91906100e2565b5090565b61010491905b808211156101005760008160009055506001016100e8565b5090565b90565b610529806101166000396000f3fe6080604052600436106100555760003560e01c80631c55ad3014610

Normally one has to wait till the transaction goes through, with ganache this is almost instant 

In [213]:
# wait till tx goes through 
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

# other way without wait only successful if mined:
# w3.eth.getTransactionReceipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0x10ea154a3dc212e1dbc57c63cc23a5758258ff0c17f8af4db802b14995e0dc31'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x2c0e4802f531edef175a38bf0283b778b9f5725972ae9fdd9ed5844aa494e880'),
 'blockNumber': 4,
 'from': '0x90f8bf6a479f320ead074411a4b0e7944ea8c9c1',
 'to': None,
 'gasUsed': 439932,
 'cumulativeGasUsed': 439932,
 'contractAddress': '0xCfEB869F69431e42cdB54A4F4f105C19C080A601',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [214]:
# the important thing from the tx receipt is the address at which 
# the contract was deployed
caddress = tx_receipt["contractAddress"]
caddress

'0xCfEB869F69431e42cdB54A4F4f105C19C080A601'

In [217]:
# double check if the gasUsed was really our estimate
blk = w3.eth.getBlock(tx_receipt["blockNumber"])
blk

AttributeDict({'number': 4,
 'hash': HexBytes('0x2c0e4802f531edef175a38bf0283b778b9f5725972ae9fdd9ed5844aa494e880'),
 'parentHash': HexBytes('0x97791403fbec12f241a2f73bf2402916e62829fed75a0e422e59f68ddfd92c55'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

In [218]:
blk["gasUsed"]

439932

### Interact with deployed contract

To get an instance of a contract at its current state we require:
* The contract address `caddress`
* Some ABI information `cabi` either from compiling the contract like done previously or reconstructed through reverse engineering of the bytecode stored in the blockchain.  

In [219]:
instance = w3.eth.contract(address=caddress,
                           abi=cabi)

In [220]:
instance.address

'0xCfEB869F69431e42cdB54A4F4f105C19C080A601'

In [221]:
instance.abi

[{'inputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'constructor'},
 {'payable': True, 'stateMutability': 'payable', 'type': 'fallback'},
 {'constant': True,
  'inputs': [],
  'name': 'getGreetbit',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'greet',
  'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'greetbit',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'greeting',
  'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': Fal

In [223]:
# view functions dont change state (just read it) and can therefore be executed 
# without performing a transaction on the blockchain
instance.functions.greet().call()

'Hello'

In [225]:
tx_hash = instance.functions.setGreeting("Hello World!").transact(
    {"from":w3.eth.accounts[0]})
tx_hash.hex()

'0x7ccda58644aaa28b18cd2f9badfed2ae56298dd0fe23d67680d2abff95b02556'

In [226]:
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [227]:
# lets check if the output is now different i.e., the value has changed 
instance.functions.greet().call()

'Hello World!'

In [228]:
tx_info = w3.eth.getTransaction(tx_hash)
tx_info

AttributeDict({'hash': HexBytes('0x7ccda58644aaa28b18cd2f9badfed2ae56298dd0fe23d67680d2abff95b02556'),
 'nonce': 4,
 'blockHash': HexBytes('0x944fa005e70340b9d8caea39535cec482d28a5b03370aa4a53971f315ab96db0'),
 'blockNumber': 6,
 'transactionIndex': 0,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xCfEB869F69431e42cdB54A4F4f105C19C080A601',
 'value': 0,
 'gas': 133737,
 'gasPrice': 20000000000,
 'input': '0xa41368620000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000c48656c6c6f20576f726c64210000000000000000000000000000000000000000',
 'v': 27,
 'r': HexBytes('0x7e6493e1cfc9cd64123434fcb82b72601f973456533843f042ab813c668f9dae'),
 's': HexBytes('0x30102041ae8f4b070cc0ef0f4312a00d955e858d04d05242524cbebcbb8f5a26')})

In [258]:
input_data = tx_info["input"][2:]
input_data

'a41368620000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000c48656c6c6f20576f726c64210000000000000000000000000000000000000000'

In [273]:
w3.toBytes(hexstr=input_data)

b'\xa4\x13hb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0cHello World!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

Lets decode the calldata input, in this case of one dynamic type parameter string:
* https://solidity.readthedocs.io/en/latest/abi-spec.html#abi

The first 4 bytes (8 hex characters) are the function signature. 
See next section for more details. 

The remaining bytes are the string parameter, seperated and aligned in three 32 bytes (256 bit) chunks. 

In [274]:
(len(input_data)-8)//64 

3

In [275]:
[ input_data[8:][i:i+64] for i in range(0,len(input_data),64) ]

['0000000000000000000000000000000000000000000000000000000000000020',
 '000000000000000000000000000000000000000000000000000000000000000c',
 '48656c6c6f20576f726c64210000000000000000000000000000000000000000',
 '']

The first parameter is the offset in bytes to the data part of the first (and only) dynamic parameter string in our case. 
This offset is computed starting from the arguments block. 

In [277]:
offset = 0x20
offset

32

After the offset the length of the string is encoded. 

In [279]:
string_length = 0x0c
string_length

12

The rest is the actual string

In [281]:
# lets decode the input data of the transaction
input_data = bytes.fromhex("48656c6c6f20576f726c6421").decode("utf-8")
print(input_data) # input data string
print("length = ",len(input_data))

Hello World!
length =  12


There is also a helper function that can decode function parameters in a more human readable form:

In [294]:
instance.decode_function_input(tx_info.input)

(<Function setGreeting(string)>, {'_greeting': 'Hello World!'})

There is also a way to directly access the storage of a contract.
> For short byte arrays, they store their data in the same slot where the length is also stored. In particular: if the data is at most 31 bytes long, it is stored in the higher-order bytes (left aligned) and the lowest-order byte stores length * 2

* https://solidity.readthedocs.io/en/latest/miscellaneous.html?highlight=storage#bytes-and-string

In [292]:
w3.eth.getStorageAt(caddress,0)

HexBytes('0x48656c6c6f20576f726c64210000000000000000000000000000000000000018')

In [285]:
w3.eth.getStorageAt(caddress,1)

HexBytes('0x00')

### Function signature / method id
The four byte *function signature* or *method id* has to be specified at the beginning of the input data. It selects the function that is invoked with the respective arguments.

In [295]:
# 4 byte function signature or method_id
len("a4136862")/2.

4.0

In [296]:
from sha3 import keccak_256
sha3_hash = keccak_256(b"setGreeting(string)").hexdigest()
method_id = sha3_hash[:8]
method_id

'a4136862'

In [297]:
instance.functions.greetbit().call()

0

In [298]:
tx_hash = instance.functions.setGreetbit(1).transact(
    {"from":w3.eth.accounts[0]})
tx_hash

HexBytes('0xb0775fd44c0c18c99278cd43dd4fc95b6e8cbf09a00eadd340779ad73a8e200d')

In [299]:
instance.functions.greetbit().call()

1

In [301]:
w3.eth.getStorageAt(caddress,1)

HexBytes('0x01')

In [302]:
tx_info = w3.eth.getTransaction(tx_hash)
tx_info

AttributeDict({'hash': HexBytes('0xb0775fd44c0c18c99278cd43dd4fc95b6e8cbf09a00eadd340779ad73a8e200d'),
 'nonce': 5,
 'blockHash': HexBytes('0xaec6c69a5c32e47b5ddfbb8c414c6046c708614dedb9a9d2e892547c2aaa423a'),
 'blockNumber': 7,
 'transactionIndex': 0,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xCfEB869F69431e42cdB54A4F4f105C19C080A601',
 'value': 0,
 'gas': 141706,
 'gasPrice': 20000000000,
 'input': '0x4f11dbea0000000000000000000000000000000000000000000000000000000000000001',
 'v': 28,
 'r': HexBytes('0x94d63414422ef729e7ed211acd0a3efce84819eefcff242404d8c31f206ce618'),
 's': HexBytes('0x0e735033961b67b1d6d4b56365ef1024932fa6e8e9d79e300eed4c7b3f34081e')})

In [304]:
from sha3 import keccak_256
sha3_hash = keccak_256(b"setGreetbit(uint256)").hexdigest()
method_id = sha3_hash[:8]
method_id

'4f11dbea'

In [305]:
len("0000000000000000000000000000000000000000000000000000000000000001")/2 * 8

256.0

You can also manually send a transaction, this should work with `geth`

In [306]:
# This should be possible with geth but is not with web3py:
cdata = b"0x4f11dbea0000000000000000000000000000000000000000000000000000000000000000"
tx_hash = w3.eth.sendTransaction({"from":w3.eth.accounts[0],"to":caddress,"data":cdata,"value":0,"gas":1_000_000})
tx_hash

HexBytes('0xdf6cb9185a6a0daaba9313c80c63c2c311c02f41644f0df31dba8642a81ee0bb')

In [307]:
instance.functions.greetbit().call()

0

You can also use the `getData()` method in the `geth` REPL console to construct the `input` data hex string. 

In [310]:
# debug i.e. trace a transaction and its operations 
w3.provider.make_request("debug_traceTransaction", [tx_hash.hex(), {}])

{'id': 162,
 'jsonrpc': '2.0',
 'result': {'gas': 31436,
  'returnValue': '',
  'structLogs': [{'depth': 0,
    'error': '',
    'gas': 973968,
    'gasCost': 26032,
    'memory': [],
    'op': 'PUSH1',
    'pc': 0,
    'stack': [],
    'storage': {}},
   {'depth': 0,
    'error': '',
    'gas': 973965,
    'gasCost': 3,
    'memory': [],
    'op': 'PUSH1',
    'pc': 2,
    'stack': ['0000000000000000000000000000000000000000000000000000000000000080'],
    'storage': {}},
   {'depth': 0,
    'error': '',
    'gas': 973962,
    'gasCost': 3,
    'memory': [],
    'op': 'MSTORE',
    'pc': 4,
    'stack': ['0000000000000000000000000000000000000000000000000000000000000080',
     '0000000000000000000000000000000000000000000000000000000000000040'],
    'storage': {}},
   {'depth': 0,
    'error': '',
    'gas': 973950,
    'gasCost': 12,
    'memory': ['0000000000000000000000000000000000000000000000000000000000000000',
     '0000000000000000000000000000000000000000000000000000000000000000',


In [309]:
# debug the old way:
#w3.providers[0].make_request("debug_traceTransaction", [tx_hash.hex(), {}])

### Fallback function

The fallback function is the function that is invoked when the method ID does not match any function or then ether is sent directly to the contract. For a contract to be able to accept ether this function has to be defined as `payable`. 

In [311]:
instance.functions.greetbit().call()

0

In [316]:
cdata = b"0xffffffff" # some random data, will be ignored anyway
tx_hash = w3.eth.sendTransaction({"from":w3.eth.accounts[0],"to":caddress,"data":cdata,"value":0})
tx_hash

HexBytes('0x24fa1e8447d84135c4e9e84390bcb2ec35c58dfec6a3be171caf7384fbfc87a6')

In [317]:
instance.functions.greetbit().call()

1

## Contract deployment and interaction (the easy way) 

Now the easy way using a small utility lib

In [318]:
pwd

'/smartcode/examples/Greeter'

In [320]:
cat Greeter.sol

pragma solidity ^0.5.12;

contract Greeter {
    string public greeting;
    uint256 public greetbit;

    //Old constructor syntax, function name same as contract name:
    //function Greeter() public {
    //new constructor syntax:
    constructor() public {
        greeting = 'Hello';
    }

    function setGreeting(string memory _greeting) public {
        greeting = _greeting;
    }

    function greet() view public returns (string memory) {
        return greeting;
    }

    function setGreetbit(uint256 _bit) public {
        greetbit = _bit;
    }

    function getGreetbit() view public returns (uint256) {
        return greetbit; 
    }

    function() external payable{
        greetbit = greetbit ^ 1;
    }
}


In [321]:
import util as util

In [328]:
util.connect(host="172.18.0.2",port=8545,poa=False)

instance = util.compile_and_deploy_contract("/smartcode/examples/Greeter/Greeter.sol")

In [329]:
instance.address

'0x9b1f7F645351AF3631a656421eD2e40f2802E6c0'

In [330]:
instance.greet()

'Hello'

In [331]:
# With the patched concise contract Factory the function
# waits till a block is mined and directly returns the tx receipt
tx_receipt = instance.setGreeting("Nihao", transact={"from": w3.eth.accounts[0]})
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x3ac69cef2fa8a8b3b887d119bd0c06aba573a816291193304c5a0b246623dd83'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xbccbf661109ab9de140a4aa8011476599738d99186276894755cbc9406ff7ed0'),
 'blockNumber': 13,
 'from': '0x90f8bf6a479f320ead074411a4b0e7944ea8c9c1',
 'to': '0x9b1f7f645351af3631a656421ed2e40f2802e6c0',
 'gasUsed': 33289,
 'cumulativeGasUsed': 33289,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [332]:
instance.greet()

'Nihao'

## Events

Events are a way to notify clients listening to new blocks about smart contract operations that have been performed.
* https://solidity.readthedocs.io/en/develop/contracts.html#events
* https://web3py.readthedocs.io/en/stable/filters.html?highlight=events
* https://web3py.readthedocs.io/en/stable/contracts.html#events

In [336]:
pwd

'/smartcode/examples'

In [337]:
cd /smartcode/examples

/smartcode/examples


In [339]:
!cat Eventer/Eventer.sol

pragma solidity ^0.5.12;

contract Eventer {

    event create(uint256 v);
    event funcall(address caller,uint256 value);
    event funcall2(address indexed caller,uint256 indexed value);
    event fallcall(address caller);

    constructor() public {
        emit create(1);
    }

    function func1(uint256 v) public {
        emit funcall(msg.sender,v);
    }

    function func2(uint256 v) public {
        emit funcall2(msg.sender,v);
    }

    function() public payable{
        emit fallcall(msg.sender);
    }

}


In [341]:
!solc --bin Eventer/Eventer.sol


======= Eventer/Eventer.sol:Eventer =======
Binary: 
608060405234801561001057600080fd5b507f780900dcfb922770b66b73546245c9d725e14dd206326f4f8f5a706976c2b61d60016040518082815260200191505060405180910390a16101ee806100586000396000f3fe6080604052600436106100295760003560e01c806316d93ade1461008e578063254e43db146100c9575b7f558048e62c62b6dedcadcb7b740131fd129cfa4019d5a5158a5befc2976d0f1d33604051808273ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390a1005b34801561009a57600080fd5b506100c7600480360360208110156100b157600080fd5b8101908080359060200190929190505050610104565b005b3480156100d557600080fd5b50610102600480360360208110156100ec57600080fd5b810190808035906020019092919050505061014b565b005b803373ffffffffffffffffffffffffffffffffffffffff167f1a7f26d3c848a605ab293135411d69990f60fa592874ca544b2f290579fbe1b860405160405180910390a350565b7fcdb58a22429d89e572d17333db36ab8054687b985157808a3aefd24cdbe175203382604051808373ffffffffffffffffffffffff

In [347]:
# create a contract instance 
einstance = util.compile_and_deploy_contract("/smartcode/examples/Eventer/Eventer.sol",concise=False)

In [348]:
einstance.address

'0xA57B8a5584442B467b4689F1144D269d096A3daF'

Now we create a listener to listen for the latest *create* events and return all of them.

In [349]:
event_filter = einstance.events.create.createFilter(fromBlock='latest')

In [350]:
event_filter.get_all_entries()

[AttributeDict({'args': AttributeDict({'v': 1}),
  'event': 'create',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x5368c7024cffec4616d3f8efea84cfd0092e893779fa018eaf067ef71df7faa4'),
  'address': '0xA57B8a5584442B467b4689F1144D269d096A3daF',
  'blockHash': HexBytes('0x1acab196fd8c12772fb9eda57b25029067c17135aa2d3eaf0eff96b7a73d46b9'),
  'blockNumber': 15})]

Now we fire a new event by creating a new transaction and keep the `tx_receipt` which we cen process later on. 

In [365]:
tx_hash = einstance.functions.func1(3).transact({"from":w3.eth.coinbase})
print(tx_hash.hex())
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

0x53e7fb648f82e06d1b16087a6983244cfde7600ad502eaf7860ad3e4ef8a4642


AttributeDict({'transactionHash': HexBytes('0x53e7fb648f82e06d1b16087a6983244cfde7600ad502eaf7860ad3e4ef8a4642'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xd40c2389c034278cc86e7a32ed0a182ed098f1753e4ee215ec139353e12849a2'),
 'blockNumber': 19,
 'from': '0x90f8bf6a479f320ead074411a4b0e7944ea8c9c1',
 'to': '0xa57b8a5584442b467b4689f1144d269d096a3daf',
 'gasUsed': 23049,
 'cumulativeGasUsed': 23049,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x53e7fb648f82e06d1b16087a6983244cfde7600ad502eaf7860ad3e4ef8a4642'),
   'blockHash': HexBytes('0xd40c2389c034278cc86e7a32ed0a182ed098f1753e4ee215ec139353e12849a2'),
   'blockNumber': 19,
   'address': '0xA57B8a5584442B467b4689F1144D269d096A3daF',
   'data': '0x00000000000000000000000090f8bf6a479f320ead074411a4b0e7944ea8c9c10000000000000000000000000000000000000000000000000000000000000003',
   'topics': [HexBytes('0xcdb58a22429d89e572d17333db36ab8054687b985157808a3

In [361]:
# concise way, not further supported with events:
#tx_receipt = einstance.func1(3,transact={"from":w3.eth.coinbase})
#tx_receipt

The `tx_receipt` contains the respective event.

In [362]:
einstance.events.funcall().processReceipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
   'value': 3}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xd7cfd5a11475ab0c74b84ae6dbfee90180a99edb942ccd5ecaaa1ea756282e90'),
  'address': '0xA57B8a5584442B467b4689F1144D269d096A3daF',
  'blockHash': HexBytes('0x282b4fb72f7e1d69dc1821ad63b7cd9c477d39cbf5261b713480cd11816cee8c'),
  'blockNumber': 18}),)

The `tx_receipt` does **not** contain the respective event.

In [364]:
einstance.events.funcall2().processReceipt(tx_receipt)

()

Now lets fire the `funcall` event again collect all events until now

In [372]:
tx_hash = einstance.functions.func1(4).transact({"from":w3.eth.coinbase})
print(tx_hash.hex())
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

0x65708f2910bc06e91910dbab5814b3eb0eeba43dc5fb26e6955656f520bfe952


AttributeDict({'transactionHash': HexBytes('0x65708f2910bc06e91910dbab5814b3eb0eeba43dc5fb26e6955656f520bfe952'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x87e3e3903940ef630e683faf9386647e55d96c5cf9956e1f3a8d49a7b0fbf3ab'),
 'blockNumber': 22,
 'from': '0x90f8bf6a479f320ead074411a4b0e7944ea8c9c1',
 'to': '0xa57b8a5584442b467b4689f1144d269d096a3daf',
 'gasUsed': 23049,
 'cumulativeGasUsed': 23049,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x65708f2910bc06e91910dbab5814b3eb0eeba43dc5fb26e6955656f520bfe952'),
   'blockHash': HexBytes('0x87e3e3903940ef630e683faf9386647e55d96c5cf9956e1f3a8d49a7b0fbf3ab'),
   'blockNumber': 22,
   'address': '0xA57B8a5584442B467b4689F1144D269d096A3daF',
   'data': '0x00000000000000000000000090f8bf6a479f320ead074411a4b0e7944ea8c9c10000000000000000000000000000000000000000000000000000000000000004',
   'topics': [HexBytes('0xcdb58a22429d89e572d17333db36ab8054687b985157808a3

In [373]:
event_filter = einstance.events.funcall.createFilter(fromBlock=0)

In [374]:
event_filter.get_all_entries()

[AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
   'value': 3}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x9d9046fe8a06372fd54fa26b935b4dffeafa05c1e94448360104999cd15dab85'),
  'address': '0xA57B8a5584442B467b4689F1144D269d096A3daF',
  'blockHash': HexBytes('0xc6014e0550b0cb1396fb9c9f4f94e5902aa0b7733182bb6b2418100c119098ec'),
  'blockNumber': 16}),
 AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
   'value': 3}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x57212378442e57905f085a67f211b1fa87943133450b92923a961d0e239b9ee6'),
  'address': '0xA57B8a5584442B467b4689F1144D269d096A3daF',
  'blockHash': HexBytes('0x304a34e45166829e079cb942d9ceaa69d1e86877cb59f78ebbf49f7e6c5266ba'),
  'blockNumber': 17}),
 AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1

In [370]:
tx_hash = w3.eth.sendTransaction({"from":w3.eth.coinbase,
                                  "to":einstance.address,
                                  "value":1,
                                  "gas":1_000_000})
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x3cd16f36d3a56a4cca3d63309e30893cdad5227939fec54b204a0f36fa3a0734'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x98e47447ae37064b875ec3ea91af961748881137de52e4c990f03524ed17c2c2'),
 'blockNumber': 21,
 'from': '0x90f8bf6a479f320ead074411a4b0e7944ea8c9c1',
 'to': '0xa57b8a5584442b467b4689f1144d269d096a3daf',
 'gasUsed': 22118,
 'cumulativeGasUsed': 22118,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x3cd16f36d3a56a4cca3d63309e30893cdad5227939fec54b204a0f36fa3a0734'),
   'blockHash': HexBytes('0x98e47447ae37064b875ec3ea91af961748881137de52e4c990f03524ed17c2c2'),
   'blockNumber': 21,
   'address': '0xA57B8a5584442B467b4689F1144D269d096A3daF',
   'data': '0x00000000000000000000000090f8bf6a479f320ead074411a4b0e7944ea8c9c1',
   'topics': [HexBytes('0x558048e62c62b6dedcadcb7b740131fd129cfa4019d5a5158a5befc2976d0f1d')],
   'type': 'mined'})],
 'status': 1,
 'logsB

In [371]:
# get the fallback function event
einstance.events.fallcall().processReceipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'}),
  'event': 'fallcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x3cd16f36d3a56a4cca3d63309e30893cdad5227939fec54b204a0f36fa3a0734'),
  'address': '0xA57B8a5584442B467b4689F1144D269d096A3daF',
  'blockHash': HexBytes('0x98e47447ae37064b875ec3ea91af961748881137de52e4c990f03524ed17c2c2'),
  'blockNumber': 21}),)

## Visibility

Generally all data stored in the blockchain is visible!
Function invocations can be restricted but be careful!

* https://solidity.readthedocs.io/en/v0.4.25/contracts.html#visibility-and-getters

In [375]:
cd /

/


In [376]:
cd /smartcode

/smartcode


In [379]:
!cat examples/Visibility/Visibility.sol

pragma solidity ^0.5.12;

contract Visibility {
    uint256 constant public cpub_int = 0x10; //set at compile time
    uint256 public pub_int;   //getter is created automatically 
    uint256 internal int_int; //no getter but available in derived contracts 
    uint256 private priv_int;

    event echange(uint256 indexed v);
    event ichange(uint256 indexed v);
    event pchange(uint256 indexed v);

    constructor() public payable {
        pub_int = 0x20;
        int_int = 0x30;
        priv_int = 0x40;
    }
    
    // Function _should_ only be called from external accounts/contracts
    // Although can also be called form this contract with this.
    function ext_change() external returns (uint256) {
        priv_int += 1;
        emit echange(priv_int);
        return priv_int;
    }
    
    // Internal functions cannot be called directly from external accounts/contracts
    // Only indirectly 
    function int_change() internal returns (uint256) {
        int_int += 1;
       

In [382]:
!solc --bin examples/Visibility/Visibility.sol


======= examples/Visibility/Visibility.sol:Visibility =======
Binary: 
60806040526020600081905550603060018190555060406002819055506103278061002b6000396000f3fe608060405234801561001057600080fd5b50600436106100625760003560e01c80630f66c3d2146100675780634c370d25146100a9578063529f246f146100c7578063ebd3b3dd146100e5578063f455a81514610103578063fcce020714610121575b600080fd5b6100936004803603602081101561007d57600080fd5b8101908080359060200190929190505050610163565b6040518082815260200191505060405180910390f35b6100b1610171565b6040518082815260200191505060405180910390f35b6100cf6101bb565b6040518082815260200191505060405180910390f35b6100ed610291565b6040518082815260200191505060405180910390f35b61010b610297565b6040518082815260200191505060405180910390f35b61014d6004803603602081101561013757600080fd5b810190808035906020019092919050505061029c565b6040518082815260200191505060405180910390f35b600060025482019050919050565b600060016002600082825401925050819055506002547f437ed1f411e2ba750c55a12da4f261a1b9f2db8c9046de9ab6119538

In [388]:
vinstance = util.compile_and_deploy_contract("./examples/Visibility/Visibility.sol",concise=False)

In [392]:
hex(vinstance.functions.pub_int().call())

'0x20'

In [393]:
hex(vinstance.functions.pub_change().call())

'0x21'

In [394]:
hex(vinstance.functions.view_priv_int(1).call())

'0x41'

In [397]:
tx_hash = vinstance.functions.pub_change().transact({"from":w3.eth.coinbase,})
tx_hash
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xd196486cef8700f2948fc5f25b295876affba6c9baee91c210f1b1e9572214a8'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xf8557f4340e5ae713024801ecd66cce7076b0f4325c5242fdc04d61b47f4ffd4'),
 'blockNumber': 26,
 'from': '0x90f8bf6a479f320ead074411a4b0e7944ea8c9c1',
 'to': '0x5f8e26facc23fa4cbd87b8d9dbbd33d5047abde1',
 'gasUsed': 43737,
 'cumulativeGasUsed': 43737,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xd196486cef8700f2948fc5f25b295876affba6c9baee91c210f1b1e9572214a8'),
   'blockHash': HexBytes('0xf8557f4340e5ae713024801ecd66cce7076b0f4325c5242fdc04d61b47f4ffd4'),
   'blockNumber': 26,
   'address': '0x5f8e26fAcC23FA4cbd87b8d9Dbbd33D5047abDE1',
   'data': '0x',
   'topics': [HexBytes('0x437ed1f411e2ba750c55a12da4f261a1b9f2db8c9046de9ab6119538e1def345'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000042')],
   'type': 'mined'}),
  

In [402]:
vinstance.events.pchange().processReceipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'v': 34}),
  'event': 'pchange',
  'logIndex': 2,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xd196486cef8700f2948fc5f25b295876affba6c9baee91c210f1b1e9572214a8'),
  'address': '0x5f8e26fAcC23FA4cbd87b8d9Dbbd33D5047abDE1',
  'blockHash': HexBytes('0xf8557f4340e5ae713024801ecd66cce7076b0f4325c5242fdc04d61b47f4ffd4'),
  'blockNumber': 26}),)

In [404]:
vinstance.events.echange().processReceipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'v': 66}),
  'event': 'echange',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xd196486cef8700f2948fc5f25b295876affba6c9baee91c210f1b1e9572214a8'),
  'address': '0x5f8e26fAcC23FA4cbd87b8d9Dbbd33D5047abDE1',
  'blockHash': HexBytes('0xf8557f4340e5ae713024801ecd66cce7076b0f4325c5242fdc04d61b47f4ffd4'),
  'blockNumber': 26}),)

In [405]:
vinstance.events.ichange().processReceipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'v': 50}),
  'event': 'ichange',
  'logIndex': 1,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xd196486cef8700f2948fc5f25b295876affba6c9baee91c210f1b1e9572214a8'),
  'address': '0x5f8e26fAcC23FA4cbd87b8d9Dbbd33D5047abDE1',
  'blockHash': HexBytes('0xf8557f4340e5ae713024801ecd66cce7076b0f4325c5242fdc04d61b47f4ffd4'),
  'blockNumber': 26}),)